In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [6]:
tf.VERSION

'1.8.0'

In [7]:
df = pd.read_csv('iris.csv')

In [8]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [9]:
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target']

In [10]:
df['target'] = df['target'].apply(int)

In [11]:
y = df['target']
x = df.drop(['target'], axis=1)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [14]:
feat_cols = []
for col in x.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [17]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=10, num_epochs=5, shuffle=True)

In [18]:
classifier = tf.estimator.DNNClassifier(hidden_units = [10, 20, 10], n_classes=3, feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/lt/qq0brrs93z944jjf5v_r23kh0000gn/T/tmpcr1_qhge', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a2681b588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
classifier.train(input_fn=input_func, steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/lt/qq0brrs93z944jjf5v_r23kh0000gn/T/tmpcr1_qhge/model.ckpt.
INFO:tensorflow:loss = 15.04228, step = 1
INFO:tensorflow:Saving checkpoints for 50 into /var/folders/lt/qq0brrs93z944jjf5v_r23kh0000gn/T/tmpcr1_qhge/model.ckpt.
INFO:tensorflow:Loss for final step: 1.4653337.


In [20]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test, batch_size=len(x_test), shuffle=False)

In [21]:
predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/lt/qq0brrs93z944jjf5v_r23kh0000gn/T/tmpcr1_qhge/model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [22]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [23]:
from sklearn.metrics import classification_report, confusion_matrix

In [25]:
print(confusion_matrix(y_test, final_preds))

[[19  0  0]
 [ 0 13  0]
 [ 0  3 10]]


In [26]:
print(classification_report(y_test, final_preds))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       0.81      1.00      0.90        13
          2       1.00      0.77      0.87        13

avg / total       0.95      0.93      0.93        45

